In [1]:
import sys
import os
from importlib import reload
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
tqdm.pandas()

%load_ext ipycache

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/lib/python3.7/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [2]:
paths = [r"/Users/Milton/Developer/Hockey/data/2007-2008 Games",
        r"/Users/Milton/Developer/Hockey/data/2008-2009 Games",
        r"/Users/Milton/Developer/Hockey/data/2009-2010 Games",
        r"/Users/Milton/Developer/Hockey/data/2010-2011 Games",
        r"/Users/Milton/Developer/Hockey/data/2011-2012 Games",
        r"/Users/Milton/Developer/Hockey/data/2013-2014 Games",
        r"/Users/Milton/Developer/Hockey/data/2014-2015 Games",
        r"/Users/Milton/Developer/Hockey/data/2015-2016 Games",
        r"/Users/Milton/Developer/Hockey/data/2016-2017 Games",
        r"/Users/Milton/Developer/Hockey/data/2017-2018 Games"]

allFiles = []

for path in paths:
    allFiles.append(glob.glob(path + "/*.csv"))

allFiles = [item for sublist in allFiles for item in sublist]

frame = pd.DataFrame()

list = []

for file in allFiles:
    df = pd.read_csv(file, index_col=None, header=0)
    list.append(df)
frame = pd.concat(list)

In [3]:
frame['ID'], frame['Matchup'] = frame['Game'].str.split('|', 1).str
frame['ID'] = frame['ID'].str.strip()
frame['Matchup'] = frame['Matchup'].str.strip()

frame['Date'] = frame['Matchup'].str.split(' ', 1).str[0].str.strip()

In [4]:
# Finding all ties (if they exist)

print(frame.loc[frame['T'] == 1])

Empty DataFrame
Columns: [i, Team, Game, GP, W, L, T, OT, P, GF, GA, S/O Win, S/O Loss, SF, SA, PPG, PP Opp, PP%, TS, PPGA, PK%, FOW, FOL, FOW%, ID, Matchup, Date]
Index: []

[0 rows x 27 columns]


In [5]:
# Dropping unused columns
frame = frame[['ID', 'Date', 'Matchup', 'Team', 'W', 'L', 'T', 'OT']]

# Adding result
def computeResult(row):
    if row['W'] == 1:
        if '@' in row['Matchup']:
            return 'A'
        else:
            return 'H'
    elif row['L'] == 1 or row['OT'] == 1:
        if '@' in row['Matchup']:
            return 'H'
        else:
            return 'A'
    
    
frame['Result'] = frame.apply(lambda row: computeResult(row), axis=1)

# Adding home team and away team
def computeHomeTeam(row):
    if '@' in row['Matchup']:
        return np.NaN
    elif 'vs' in row['Matchup']:
        return row['Team']
    
def computeAwayTeam(row):
    if '@' in row['Matchup']:
        return row['Team']
    elif 'vs' in row['Matchup']:
        return np.NaN

frame['HomeTeam'] = frame.apply(lambda row: computeHomeTeam(row), axis=1)
frame['AwayTeam'] = frame.apply(lambda row: computeAwayTeam(row), axis=1)

In [6]:
# Finalize scores data

frame = frame.groupby(['ID']).last().reset_index()
# Dropping unused columns
frame = frame[['ID', 'Date', 'HomeTeam', 'AwayTeam', 'Result']]
          
    
# Ensure data is still valid + Percentage of Home Wins
numberOfMatches = frame.shape[0]
numberOfHomeWins = frame.loc[frame['Result'] == 'H'].shape[0]

print('Total number of matches: {}'.format(numberOfMatches))
print('Percentage of home wins: {:.2f}%'.format(float(numberOfHomeWins) / numberOfMatches * 100))

Total number of matches: 12341
Percentage of home wins: 54.65%


In [7]:
frame['HomeTeam'] = frame['HomeTeam'].replace(['Phoenix Coyotes', 'Arizona Coyotes'], 'Coyotes')
frame['AwayTeam'] = frame['AwayTeam'].replace(['Phoenix Coyotes', 'Arizona Coyotes'], 'Coyotes')

# Save to csv
frame.to_csv('Full_Match_Results_2007-2018.csv')
display(frame)

,ID,Date,HomeTeam,AwayTeam,Result
0,https://www.nhl.com/gamecenter/2007020001,2007/09/29,Los Angeles Kings,Anaheim Ducks,H
1,https://www.nhl.com/gamecenter/2007020002,2007/09/30,Anaheim Ducks,Los Angeles Kings,H
2,https://www.nhl.com/gamecenter/2007020003,2007/10/03,Carolina Hurricanes,Montréal Canadiens,A
3,https://www.nhl.com/gamecenter/2007020004,2007/10/03,Detroit Red Wings,Anaheim Ducks,H
4,https://www.nhl.com/gamecenter/2007020005,2007/10/03,Toronto Maple Leafs,Ottawa Senators,A
5,https://www.nhl.com/gamecenter/2007020006,2007/10/03,Colorado Avalanche,Dallas Stars,H
6,https://www.nhl.com/gamecenter/2007020007,2007/10/04,New York Rangers,Florida Panthers,H
7,https://www.nhl.com/gamecenter/2007020008,2007/10/04,Ottawa Senators,Toronto Maple Leafs,H
8,https://www.nhl.com/gamecenter/2007020009,2007/10/04,Tampa Bay Lightning,New Jersey Devils,H
9,https://www.nhl.com/gamecenter/2007020010,2007/10/04,Nashville Predators,Colorado Avalanche,H


In [8]:
def standardizeTeamNames(dataFrame):
    dataFrame['teamName'] = dataFrame['teamName'].replace(['Phoenix Coyotes', 'Arizona Coyotes'], 'Coyotes')
    dataFrame['teamName'] = dataFrame['teamName'].replace(['Montreal Canadiens'], 'Montréal Canadiens')
    dataFrame['teamName'] = dataFrame['teamName'].replace(['St Louis Blues'], 'St. Louis Blues')
    

In [9]:
# Creating aggregated standings file

import json
from pandas.io.json import json_normalize


# Adding standings data from Corsica
path = r"/Users/Milton/Developer/Hockey/data/Standings All"

jsonList = []

for file in glob.glob(path + "/*.json"):
    with open(file, 'rb') as infile:
        rawJson = json.load(infile)
        standingsJson = rawJson['teams']
        for team in standingsJson:
            team['endDate'] = rawJson['params']['endDate']
        jsonList.extend(standingsJson)

standings = pd.DataFrame(jsonList)
    
standardizeTeamNames(standings)

standings.to_csv('Raw_Standings_2007-2018.csv')
print(standings.shape[0])

55442


In [10]:
# Adding standings data from NHL
path = r"/Users/Milton/Developer/Hockey/data/Standings NHL"

standingsList = []

nhlStandingsFrame = pd.DataFrame()

for file in glob.glob(path + "/*.csv"):
    with open(file, 'rb') as infile:
        df = pd.read_csv(file, index_col=None, header=0)
        df['endDate'] = os.path.basename(file).split('.')[0]
        standingsList.append(df)
        
nhlStandingsFrame = pd.concat(standingsList)

# Rename 'Team' Column
nhlStandingsFrame.rename(columns={'Team': 'teamName'}, inplace=True)
standardizeTeamNames(nhlStandingsFrame)

# Combining two standings dataframes
standings = pd.merge(standings, nhlStandingsFrame, how='left', left_on=['endDate', 'teamName'], right_on=['endDate', 'teamName'])
print(standings.sort_values('date')[['teamName', 'endDate']])

                    teamName     endDate
40749  Philadelphia Flyers    2010-4-1  
40769  Coyotes                2010-4-1  
40768  Boston Bruins          2010-4-1  
40767  Buffalo Sabres         2010-4-1  
40766  Carolina Hurricanes    2010-4-1  
40765  Columbus Blue Jackets  2010-4-1  
40764  Calgary Flames         2010-4-1  
40763  Chicago Blackhawks     2010-4-1  
22608  Columbus Blue Jackets  2014-4-1  
40770  Anaheim Ducks          2010-4-1  
22609  Carolina Hurricanes    2014-4-1  
22611  Boston Bruins          2014-4-1  
22612  Coyotes                2014-4-1  
22613  Anaheim Ducks          2014-4-1  
40762  Colorado Avalanche     2010-4-1  
40761  Dallas Stars           2010-4-1  
40760  Detroit Red Wings      2010-4-1  
40759  Edmonton Oilers        2010-4-1  
40758  Florida Panthers       2010-4-1  
22610  Buffalo Sabres         2014-4-1  
22607  Calgary Flames         2014-4-1  
22606  Chicago Blackhawks     2014-4-1  
22605  Colorado Avalanche     2014-4-1  
22585  Winnipeg 

In [11]:
# Remove games played before and on November 1 of each season because of lack of sample size
import datetime
    
def dropEarlyGames(row):
    year = row.split("/", 1)[0]
    currentDate = datetime.datetime.strptime(row, '%Y/%m/%d')
    startDate = datetime.datetime(int(year), 9, 1)
    endDate = datetime.datetime(int(year), 11, 1)
    
    if startDate <= currentDate <= endDate:
        return False
    else: 
        return True
    
# frame = frame[frame['Date'].apply(lambda row: dropEarlyGames(row))]
    
frame.drop(frame[(frame['HomeTeam'] == 'Atlanta Thrashers') | (frame['AwayTeam'] == 'Atlanta Thrashers')].index, inplace=True)


In [12]:
# Fetch standings given team, date in 2007/09/29

def teamStandings(team, date):
#     # Check for different team names
#     adjustedTeam = team
#     if team == 'Phoenix Coyotes':
#         adjustedTeam = 'Arizona Coyotes'
#     elif team == 'Montréal Canadiens':
#         adjustedTeam = 'Montreal Canadiens'
#     elif team == 'St. Louis Blues':
#         adjustedTeam = 'St Louis Blues'
    
    # Convert date to datetime
    date = datetime.datetime.strptime(date, '%Y/%m/%d')
    
    # Get previous date
    date = date - datetime.timedelta(1)
    
    standingsDate = date.strftime('%Y-%-m-%-d')
    
    return standings.loc[(standings['endDate'] == standingsDate) & (standings['teamName'] == team)].head(1)

teamStandings('Los Angeles Kings', '2007/09/29')

,cA,cA60,cF,cF%,cF60,cplusminus,dFSh%,dFSv%,dPDO,date,...,SA,PPG,PP Opp,PP%,TS,PPGA,PK%,FOW,FOL,FOW%


In [13]:
frame.loc[frame['Date'] == '2016/01/23']

,ID,Date,HomeTeam,AwayTeam,Result
9318,https://www.nhl.com/gamecenter/2015020709,2016/01/23,Pittsburgh Penguins,Vancouver Canucks,H
9319,https://www.nhl.com/gamecenter/2015020710,2016/01/23,San Jose Sharks,Minnesota Wild,H
9320,https://www.nhl.com/gamecenter/2015020711,2016/01/23,Boston Bruins,Columbus Blue Jackets,H
9321,https://www.nhl.com/gamecenter/2015020712,2016/01/23,Toronto Maple Leafs,Montréal Canadiens,A
9322,https://www.nhl.com/gamecenter/2015020713,2016/01/23,Detroit Red Wings,Anaheim Ducks,A
9323,https://www.nhl.com/gamecenter/2015020714,2016/01/23,Florida Panthers,Tampa Bay Lightning,H
9324,https://www.nhl.com/gamecenter/2015020715,2016/01/23,Winnipeg Jets,New Jersey Devils,A
9326,https://www.nhl.com/gamecenter/2015020717,2016/01/23,Dallas Stars,Colorado Avalanche,A
9327,https://www.nhl.com/gamecenter/2015020718,2016/01/23,Coyotes,Los Angeles Kings,H
9328,https://www.nhl.com/gamecenter/2015020719,2016/01/23,Edmonton Oilers,Nashville Predators,A


In [14]:
oddsFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Odds 2007-2018 Raw.csv')
        
def findPredictionAndResult(row):
    result = frame.loc[(frame['HomeTeam'].str.contains(row['HomeTeam'])) & (frame['AwayTeam'].str.contains(row['AwayTeam'])) & (frame['Date'] == row['Date'])]
    if not result.empty:
        result = result.iloc[0]['Result']
    else:
        result = float('NaN')
        return row

    row['Result'] = 1 if result == 'H' else 0
    homePayout = row['HomeOdds']
    awayPayout = row['AwayOdds']
    vegasPrediction = None
    if homePayout < awayPayout:
        vegasPrediction = 1
    elif homePayout > awayPayout:
        vegasPrediction = 0
    else:
        vegasPrediction = float('NaN')
    row['VegasPrediction'] = vegasPrediction
    return row
    
oddsFrame = oddsFrame.progress_apply(findPredictionAndResult, axis=1)

100%|██████████| 12185/12185 [04:02<00:00, 50.31it/s]


In [15]:
oddsFrame = oddsFrame.dropna()
display(oddsFrame.loc[(oddsFrame['Date'] == '2007/09/29')])

,AwayOdds,AwayTeam,Date,HomeOdds,HomeTeam,Result,VegasPrediction
5348,0.813008,Anaheim,2007/09/29,1.13,Los Angeles,1.0,0.0


In [16]:
def selectGamesUpToDate(row, gameDate):
    currentDate = datetime.datetime.strptime(row, '%Y/%m/%d')
    endDate = datetime.datetime.strptime(gameDate, '%Y/%m/%d')

    startYear = None
    if endDate.month >= 9:
        startYear = endDate.year
    elif endDate.month <= 4:
        startYear = endDate.year - 1
    else:
        print(currentDate.month)
    
    startDate = datetime.datetime(startYear, 9, 1)
    
    if startDate <= currentDate < endDate:
        return True
    else: 
        return False

def selectTeam(row, teamName):
    if row['HomeTeam'] in teamName or row['AwayTeam'] in teamName:
        return True
    else:
        return False
    
def calculateFavorites(teamName, gameDate):
#     team = oddsFrame.loc[(oddsFrame['HomeTeam'] in teamName) | (oddsFrame['AwayTeam'] in teamName)]
#     print(type(oddsFrame[oddsFrame['HomeTeam'].str in teamName]))
    team = oddsFrame.loc[oddsFrame.apply(lambda row: selectTeam(row, teamName), axis=1)]
#     team = oddsFrame[team['Date'].apply(lambda row: selectGamesUpToDate(row, gameDate))]
    team = team[team['Date'].apply(lambda row: selectGamesUpToDate(row, gameDate))]
    results = team[['Result']].values.flatten()
    vegasPredictions = team[['VegasPrediction']].values.flatten()
    
    if team.empty: # First game of the season
        return float('NaN')
        
    return sum(results == vegasPredictions) / float(len(team))
    
def selectGames(row):
    homeTeamFavorite = calculateFavorites(row['HomeTeam'], row['Date'])
    awayTeamFavorite = calculateFavorites(row['AwayTeam'], row['Date'])
    print(homeTeamFavorite)
    print(awayTeamFavorite)
    row['FavoritesW%'] = homeTeamFavorite * awayTeamFavorite
    
    return row
    
frame = frame.progress_apply(selectGames, axis=1)

  0%|          | 2/12013 [00:00<1:18:44,  2.54it/s]

nan
nan


  0%|          | 3/12013 [00:01<1:39:23,  2.01it/s]

nan
nan


  0%|          | 4/12013 [00:02<2:00:22,  1.66it/s]

...

0.6419753086419753
0.6049382716049383


100%|█████████▉| 12012/12013 [2:28:43<00:00,  1.38it/s]

0.55
0.5569620253164557


100%|██████████| 12013/12013 [2:28:44<00:00,  1.39it/s]

0.5308641975308642
0.5375
0.5063291139240507
0.6049382716049383
0.5569620253164557
0.5802469135802469


In [29]:
def selectGamesUpToDate(row, gameDate):
    currentDate = datetime.datetime.strptime(row, '%Y/%m/%d')
    endDate = datetime.datetime.strptime(gameDate, '%Y/%m/%d')

    startYear = None
    if endDate.month >= 9:
        startYear = endDate.year
    elif endDate.month <= 4:
        startYear = endDate.year - 1
    else:
        print(currentDate.month)
    
    startDate = datetime.datetime(startYear, 9, 1)
    
#     import ipdb; ipdb.set_trace()
    
    if startDate <= currentDate < endDate:
        return True
    else: 
        return False

def selectTeam(row, teamName):
    if row['HomeTeam'] in teamName or row['AwayTeam'] in teamName:
        return True
    else:
        return False
    
def calculateFavorites(teamName, gameDate):
#     team = oddsFrame.loc[(oddsFrame['HomeTeam'] in teamName) | (oddsFrame['AwayTeam'] in teamName)]
#     print(type(oddsFrame[oddsFrame['HomeTeam'].str in teamName]))
    team = oddsFrame.loc[oddsFrame.apply(lambda row: selectTeam(row, teamName), axis=1)]
#     team = oddsFrame[team['Date'].apply(lambda row: selectGamesUpToDate(row, gameDate))]
    team = team[team['Date'].apply(lambda row: selectGamesUpToDate(row, gameDate))]
    display(team)
    results = team[['Result']].values.flatten()
    vegasPredictions = team[['VegasPrediction']].values.flatten()
    
    if team.empty: # First game of the season
        return float('NaN')
        
    return sum(results == vegasPredictions) / float(len(team))
    
def selectGames(row):
    homeTeamFavorite = calculateFavorites(row['HomeTeam'], row['Date'])
    awayTeamFavorite = calculateFavorites(row['AwayTeam'], row['Date'])
    print(homeTeamFavorite)
    print(awayTeamFavorite)
    row['FavoritesW%'] = homeTeamFavorite * awayTeamFavorite
    
    return row

frame.tail(1).apply(selectGames, axis=1)

,AwayOdds,AwayTeam,Date,HomeOdds,HomeTeam,Result,VegasPrediction
171,1.400000,New Jersey,2018/03/20,0.645161,San Jose,1.0,1.0
301,1.370000,Boston,2017/11/18,0.657895,San Jose,0.0,1.0
368,1.030000,San Jose,2017/12/31,0.877193,Dallas,1.0,1.0
377,0.980392,San Jose,2017/10/26,0.925926,Boston,1.0,1.0
476,1.490000,San Jose,2018/01/21,0.606061,Anaheim,0.0,1.0
490,1.030000,Vegas,2018/02/08,0.877193,San Jose,0.0,1.0
636,0.781250,San Jose,2017/11/22,1.160000,Coyotes,0.0,0.0
728,1.050000,Toronto,2017/10/30,0.862069,San Jose,1.0,1.0
1175,1.130000,San Jose,2018/03/16,0.800000,Calgary,0.0,1.0
1228,1.200000,San Jose,2018/02/04,0.751880,Carolina,0.0,1.0


,AwayOdds,AwayTeam,Date,HomeOdds,HomeTeam,Result,VegasPrediction
88,0.934579,Dallas,2017/12/27,0.970874,Minnesota,1.0,0.0
298,1.020000,Minnesota,2017/11/18,0.884956,Washington,1.0,1.0
385,1.160000,New York Islanders,2017/10/26,0.781250,Minnesota,1.0,1.0
488,2.010000,Coyotes,2018/02/08,0.444444,Minnesota,0.0,1.0
626,0.671141,Minnesota,2017/11/22,1.350000,Buffalo,0.0,0.0
820,1.060000,Montréal,2017/11/02,0.854701,Minnesota,1.0,1.0
832,1.520000,Edmonton,2018/04/02,0.595238,Minnesota,1.0,1.0
1067,1.350000,Philadelphia,2017/11/14,0.671141,Minnesota,1.0,1.0
1178,1.550000,Minnesota,2018/03/16,0.581395,Vegas,0.0,1.0
1522,0.636943,Minnesota,2018/03/01,1.420000,Coyotes,1.0,0.0


0.5569620253164557
0.5802469135802469


,AwayOdds,AwayTeam,Date,HomeOdds,HomeTeam,Result,VegasPrediction
171,1.400000,New Jersey,2018/03/20,0.645161,San Jose,1.0,1.0
301,1.370000,Boston,2017/11/18,0.657895,San Jose,0.0,1.0
368,1.030000,San Jose,2017/12/31,0.877193,Dallas,1.0,1.0
377,0.980392,San Jose,2017/10/26,0.925926,Boston,1.0,1.0
476,1.490000,San Jose,2018/01/21,0.606061,Anaheim,0.0,1.0
490,1.030000,Vegas,2018/02/08,0.877193,San Jose,0.0,1.0
636,0.781250,San Jose,2017/11/22,1.160000,Coyotes,0.0,0.0
728,1.050000,Toronto,2017/10/30,0.862069,San Jose,1.0,1.0
1175,1.130000,San Jose,2018/03/16,0.800000,Calgary,0.0,1.0
1228,1.200000,San Jose,2018/02/04,0.751880,Carolina,0.0,1.0


,AwayOdds,AwayTeam,Date,HomeOdds,HomeTeam,Result,VegasPrediction
88,0.934579,Dallas,2017/12/27,0.970874,Minnesota,1.0,0.0
298,1.020000,Minnesota,2017/11/18,0.884956,Washington,1.0,1.0
385,1.160000,New York Islanders,2017/10/26,0.781250,Minnesota,1.0,1.0
488,2.010000,Coyotes,2018/02/08,0.444444,Minnesota,0.0,1.0
626,0.671141,Minnesota,2017/11/22,1.350000,Buffalo,0.0,0.0
820,1.060000,Montréal,2017/11/02,0.854701,Minnesota,1.0,1.0
832,1.520000,Edmonton,2018/04/02,0.595238,Minnesota,1.0,1.0
1067,1.350000,Philadelphia,2017/11/14,0.671141,Minnesota,1.0,1.0
1178,1.550000,Minnesota,2018/03/16,0.581395,Vegas,0.0,1.0
1522,0.636943,Minnesota,2018/03/01,1.420000,Coyotes,1.0,0.0


0.5569620253164557
0.5802469135802469


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,PDO,PENDiff,SF%,SDiff,Sh%,Sv,FOW%,W%,FavoritesW%,Result
12340,https://www.nhl.com/gamecenter/2017021271,2018/04/07,San Jose Sharks,Minnesota Wild,4.54,-0.88521,0.067829,4.15,-0.866669,-0.089346,...,-0.84,58.0,2.87,3.57,-0.815267,-0.02,1.4,0.012346,0.323175,A


In [18]:
frame = frame.dropna()

,ID,Date,HomeTeam,AwayTeam,Result,FavoritesW%
1,https://www.nhl.com/gamecenter/2007020002,2007/09/30,Anaheim Ducks,Los Angeles Kings,H,0.000000
7,https://www.nhl.com/gamecenter/2007020008,2007/10/04,Ottawa Senators,Toronto Maple Leafs,H,1.000000
20,https://www.nhl.com/gamecenter/2007020021,2007/10/06,Edmonton Oilers,Philadelphia Flyers,H,0.000000
21,https://www.nhl.com/gamecenter/2007020022,2007/10/06,Toronto Maple Leafs,Montréal Canadiens,H,0.000000
22,https://www.nhl.com/gamecenter/2007020023,2007/10/06,Ottawa Senators,New York Rangers,H,1.000000
23,https://www.nhl.com/gamecenter/2007020024,2007/10/06,New York Islanders,Buffalo Sabres,H,0.000000
25,https://www.nhl.com/gamecenter/2007020026,2007/10/06,Florida Panthers,New Jersey Devils,A,1.000000
26,https://www.nhl.com/gamecenter/2007020027,2007/10/06,Pittsburgh Penguins,Anaheim Ducks,H,0.000000
28,https://www.nhl.com/gamecenter/2007020029,2007/10/06,Nashville Predators,Dallas Stars,H,1.000000
29,https://www.nhl.com/gamecenter/2007020030,2007/10/06,Minnesota Wild,Columbus Blue Jackets,H,0.000000


In [19]:
frame = frame[frame['Date'].apply(lambda row: dropEarlyGames(row))]

In [20]:
# %%cache finalDatasetCache.pkl frame
# Calculating features (Home team - Away Team)

# Corsi For Percentage (CF%) (cF%) - Percentage of all shots taken by a team
# Corsi Shooting Percentage (CSh%) (gF/cF) - Percentage of shots taken by a team that are goals
# Corsi Save Percentage (CSv%) (1-gA/cA) - Percentage of shots against a team that are saves
# Fenwick For Percentage (FF%) (fF%) - Percentage of all unblocked shots that are taken by a team
# Fenwick Shooting Percentage (FSh%) (fSh%, gF/fF) - Percentage of unblocked shots taken by a team that are goals
# Fenwick Save Percentage (FSv%) (fSv%, 1-gA/fA) - Percentage of unblocked against a team that are saves
# Goals Difference (GDiff) (gF60-gA60) - Goals For per 60 minus Goals Against per 60
# Goals For Percentage (GF%) (gF%) - Percentage of all goals that are scored by a team
# PDO (PDO) (pDO) - Shooting percentage + Save percentage
# Penalty Differential (PENDiff) (pEND-pENT) - Difference of penalities drawn and taken
# Shots on Goal For Percentage (SF%) (sF%) - Percentage of all shots on goal that are taken by a team
# Shots on Goal Difference (SDiff) (sF60-sA60) - Shots on goal per 60 minus shots on goal against a team  per 60
# Shooting Percentage (Sh%) (gF/sF) - Percentage of shots on goal taken by a team that are goals
# Save Percentage (Sv%) (sv%) - Percentage of shots on goal against that are saves
# Faceoffs Percentage (FOW%) - Percentage of faceoffs won by a team
# Win Percentage (W%) - Percentage of max possible points gained

### May add later
### Scoring Chance Difference (SCF60-SCA60) - Scoring chances for per 60 minus scoring chances against per 60
### Scoring Chances For Percentage (SCF%) - Percentage of all scoring chances that are taken by a team

def value(frame, key):
    return frame.iloc[0][key].astype(float)

def intValue(frame, key):
    return frame.iloc[0][key]

def addFeatures(row):
    home = teamStandings(row['HomeTeam'], row['Date'])
    away = teamStandings(row['AwayTeam'], row['Date'])

    row['CF%'] = value(home, 'cF%') - value(away, 'cF%')
    corsiShooting = (value(home, 'gF') / value(home, 'cF')) - (value(away, 'gF') / value(away, 'cF'))
    row['CSh%'] = corsiShooting * 100
    corsiSave = (1 - (value(home, 'gA') / value(home, 'cA'))) - (1 - (value(away, 'gA') / value(away, 'cA')))
    row['CSv%'] = corsiSave * 100
    row['FF%'] = value(home, 'fF%') - value(away, 'fF%')
    fenwickShooting = (value(home, 'gF') / value(home, 'fF')) - (value(away, 'gF') / value(away, 'fF'))
    row['FSh%'] = fenwickShooting * 100
    fenwickSave = (1 - (value(home, 'gA') / value(home, 'fA'))) - (1 - (value(away, 'gA') / value(away, 'fA')))
    row['FSv%'] = fenwickSave * 100
    row['GDiff'] = (value(home, 'gF60') - value(home, 'gA60')) - (value(away, 'gF60') - value(away, 'gA60'))
    row['GF%'] = value(home, 'gF%') - value(away, 'gF%')
    row['PDO'] = value(home, 'pDO') - value(away, 'pDO')
    row['PENDiff'] = (value(home, 'pEND') - value(home, 'pENT')) - (value(away, 'pEND') - value(away, 'pENT'))
    row['SF%'] = value(home, 'sF%') - value(away, 'sF%')
    row['SDiff'] = (value(home, 'sF60') - value(home, 'sA60')) - (value(away, 'sF60') - value(away, 'sA60'))
    shootingPercentage = (value(home, 'gF') / value(home, 'sF')) - (value(away, 'gF') / value(away, 'sF'))
    row['Sh%'] = shootingPercentage * 100
    row['Sv'] = value(home, 'sv%') - value(away, 'sv%')
    row['FOW%'] = value(home, 'FOW%') - value(away, 'FOW%')
    winPercentage = (intValue(home, 'W') / intValue(home, 'GP')) - (intValue(away, 'W') / intValue(away, 'GP'))
    row['W%'] = np.float64(winPercentage)
    return row

frame = frame.progress_apply(addFeatures, axis=1)


100%|██████████| 10343/10343 [08:16<00:00, 20.83it/s]


In [21]:
# oddsFrame.to_csv('Bets_Results_2007-2018.csv')

In [22]:
# frame = oddsFrame

# oddsFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Odds 2007-2018 Raw.csv')

# oddsFrame
display(frame['FavoritesW%'].max())


0.9090909090909091

In [23]:
# display(oddsFrame.loc[oddsFrame['Date'] == '2007/11/02'])

In [25]:
frame = frame[['ID', 
               'Date', 
               'HomeTeam', 
               'AwayTeam', 
               'CF%', 
               'CSh%',
               'CSv%',
               'FF%',
               'FSh%',
               'FSv%',
               'GDiff',
               'GF%',
               'PDO',
               'PENDiff',
               'SF%',
               'SDiff',
               'Sh%',
               'Sv',
               'FOW%',
               'W%',
               'FavoritesW%',
               'Result']]


frame.to_csv('Favorites_Full_Dataset_Games_Standings_2007-2018.csv')
print(frame.shape[0])

print(frame[frame.isnull().any(axis=1)])

10343
Empty DataFrame
Columns: [ID, Date, HomeTeam, AwayTeam, CF%, CSh%, CSv%, FF%, FSh%, FSv%, GDiff, GF%, PDO, PENDiff, SF%, SDiff, Sh%, Sv, FOW%, W%, FavoritesW%, Result]
Index: []

[0 rows x 22 columns]


In [28]:
frame['FavoritesW%'].median()

display(frame.tail(1000)['FavoritesW%'].max())

0.5212620027434842